# Machine Learning for Seagrass Extent Prediction

### Remember to run all code cells from the top in sequence


In [1]:
from pystac.client import Client
from odc.stac import load
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import joblib
from matplotlib import colors


from utils import scale, apply_masks, do_prediction, calculate_band_indices

### Input your details

1. Your initials  
2. Date  
3. Site

In [1]:
filename = your_initials+'_'+site+'_'+date
print(filename)

In [4]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

#### Input the bottom left and top right coordinates of your site. You can find these in QGIS / Googlemaps or similar options


#### Input the year of analysis (noting the year's available are between 2017 and 2024)

In [6]:
datetime="2024"

#### Well done, you have completed the configuration steps. Now you can click the fast forward button at the top to run the code and generate a prediction.

In [7]:
items = client.search(
    collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox
).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [8]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 312MB
Dimensions:      (y: 2007, x: 4092, time: 1)
Coordinates:
  * y            (y) float64 16kB -2.033e+06 -2.033e+06 ... -2.053e+06
  * x            (x) float64 33kB 3.15e+06 3.15e+06 ... 3.191e+06 3.191e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    red          (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 33MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 33MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    swir22       (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    coastal      (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    rededge1     (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    rededge2     (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    rededge3     (time, y, x) uint16 16MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>

In [9]:
scaled_data = scale(data)
indices = calculate_band_indices(scaled_data)

# TODO: split masks into separate functions
masked_data = apply_masks(scaled_data)

masked_data

<xarray.Dataset> Size: 2GB
Dimensions:      (time: 1, y: 2007, x: 4092)
Coordinates:
  * y            (y) float64 16kB -2.033e+06 -2.033e+06 ... -2.053e+06
  * x            (x) float64 33kB 3.15e+06 3.15e+06 ... 3.191e+06 3.191e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    red          (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    blue         (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    green        (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 33MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 33MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    ...           ...
    ndvi         (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    ndwi         (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    b_g          (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    b_r          (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    mci          (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>
    ndci         (time, y, x) float64 66MB dask.array<chunksize=(1, 2007, 2048), meta=np.ndarray>

In [3]:
masked_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

## Run predictions over our region

In [11]:
model = joblib.load("models/model-geomad-joined-data-rf-04032025.model")

In [12]:
predicted_da = do_prediction(masked_data, model).astype(np.float32)
predicted_da

<xarray.DataArray (y: 2007, x: 4092, time: 1)> Size: 33MB
array([[[nan],
        [nan],
        [nan],
        ...,
        [ 8.],
        [ 8.],
        [ 8.]],

       [[nan],
        [nan],
        [nan],
        ...,
        [ 8.],
        [ 8.],
        [ 8.]],

       [[nan],
        [nan],
        [nan],
        ...,
...
        ...,
        [ 8.],
        [ 8.],
        [ 8.]],

       [[ 8.],
        [ 8.],
        [ 8.],
        ...,
        [ 8.],
        [ 8.],
        [ 8.]],

       [[ 8.],
        [ 8.],
        [ 8.],
        ...,
        [ 8.],
        [ 8.],
        [ 8.]]], dtype=float32)
Coordinates:
  * y            (y) float64 16kB -2.033e+06 -2.033e+06 ... -2.053e+06
  * x            (x) float64 33kB 3.15e+06 3.15e+06 ... 3.191e+06 3.191e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01

In [13]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#00FFFFFF"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

In [4]:
predicted_da.odc.explore(cmap=c_map)

In [2]:
predicted_da.odc.write_cog(f"prediction_{filename}.tiff", overwrite=True)

# Notebook completed

1. Now you can download the new .tiff file from the side panel and open in QGIS. You can change the colours of each class in the next activity. 